In [20]:
import keyring        
import pymssql
from contextlib import contextmanager
import pandas as pd

In [21]:
@contextmanager
def GetDBConnection():
    server = 'ITSMDB'
    database = 'BSC_4_0'
    username = 'ingham-county\\'+keyring.get_password("VPN_Ingham-County",'USER')
    password = keyring.get_password("VPN_Ingham-County",keyring.get_password("VPN_Ingham-County",'USER'))

    # Connect to the database
    conn = pymssql.connect(server, username, password, database,as_dict=True)

    cursor = conn.cursor()
    try:
        yield cursor
    finally:
        conn.close()

In [22]:
with GetDBConnection() as cursor:
    sql = """
    SELECT [Id] FROM [BSC_4_0].[dbo].[AbpUsers] where UserName like '%Rudloff%'
    """
    cursor.execute(sql)
    userid = cursor.fetchone()['Id']
userid

8908

In [23]:
def findTicketsAssigned(uid):
    with GetDBConnection() as cursor:
        sql = """
        SELECT
        [Id],
        [Title],
        [Description]
        FROM [Tickets] where AgentId = %s
        """
        cursor.execute(sql, (uid))
        return pd.DataFrame(cursor.fetchall())
my_tickets = findTicketsAssigned(userid)
len(my_tickets)
file_path = 'my_tickets.csv'
my_tickets.to_csv(file_path, index=False)

In [24]:
def findComments(uid):
    with GetDBConnection() as cursor:
        sql = """
        SELECT
        [TicketId],
        [Body]
        FROM [Comments] where CreatorUserId = %s
        """
        cursor.execute(sql, (uid))
        return pd.DataFrame(cursor.fetchall())
my_comments = findComments(userid)
len(my_comments)
file_path = 'my_comments.csv'
my_comments.to_csv(file_path, index=False)

In [25]:
def SearchComments(keyword):
    with GetDBConnection() as cursor:
        sql = """
        SELECT
        [TicketId],
        [Body]
        FROM [Comments]
        where Body like %s
        """
        cursor.execute(sql, keyword)
        return cursor.fetchall()

In [26]:
with GetDBConnection() as cursor:
    sql = """
    SELECT
    [Id],
    [Title],
    [Description]
    FROM [Tickets]
    """
    cursor.execute(sql)
    all_tickets =  cursor.fetchall()

In [27]:
len(all_tickets)

41082

In [28]:
df = pd.DataFrame(all_tickets)

In [29]:
# Save DataFrame to a CSV file
file_path = 'all_tickets.csv'
df.to_csv(file_path, index=False)

In [30]:
with GetDBConnection() as cursor:
    sql = """
    SELECT
    [TicketId],
    [Body]
    FROM [Comments]
    """
    cursor.execute(sql)
    data = cursor.fetchall()

In [31]:
len(data)

108388

In [32]:
import pandas as pd
df = pd.DataFrame(data)
file_path = 'comments.csv'
df.to_csv(file_path, index=False)